In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv('survival.csv',sep= ',')
df.head()

,studyid,site,gender,age,agecat,race_ethnicity,maritalstatus,education,lowhealthliteracy,employment status,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,10000,Massachusetts,Male,60,>=55-<65,NHW,Married,College Graduate,No,Employed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10007,Massachusetts,Male,61,>=55-<65,NHW,Married,Some college,Yes,Unemployed/retired,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10012,Massachusetts,Male,56,>=55-<65,NHW,Not Married,High School or less,Yes,Unemployed/retired,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10013,Massachusetts,Male,48,<55,NHW,Not Married,High School or less,No,Employed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10020,Massachusetts,Male,49,<55,NHW,Not Married,High School or less,No,Unemployed/retired,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Columns: 256 entries, studyid to Unnamed: 255
dtypes: float64(206), int64(4), object(46)
memory usage: 4.0+ MB


## Data Cleaning

In [5]:
target = 'died_2_year'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
df = df.replace('nan',np.nan,regex=True)

#df

### Remove outliers

In [7]:
# Drop the column outliers as it is unrelated to the dependent variable
outliers = ['studyid','agecat']
df = df.drop(outliers,axis=1)
df.head()

,site,gender,age,race_ethnicity,maritalstatus,education,lowhealthliteracy,employment status,healthinsurance,strength_comfort_religion,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,Massachusetts,Male,60,NHW,Married,CollegeGraduate,No,Employed,No,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Massachusetts,Male,61,NHW,Married,Somecollege,Yes,Unemployed/retired,No,Little/Some,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Massachusetts,Male,56,NHW,NotMarried,HighSchoolorless,Yes,Unemployed/retired,Yes,Greatdeal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Massachusetts,Male,48,NHW,NotMarried,HighSchoolorless,No,Employed,No,Little/Some,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Massachusetts,Male,49,NHW,NotMarried,HighSchoolorless,No,Unemployed/retired,No,Little/Some,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,300)
df.info()

number of bad columns: 194
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Data columns (total 60 columns):
site                                2068 non-null object
gender                              2053 non-null object
age                                 2068 non-null int64
race_ethnicity                      2059 non-null object
maritalstatus                       2068 non-null object
education                           2067 non-null object
lowhealthliteracy                   2068 non-null object
employment status                   2068 non-null object
healthinsurance                     2068 non-null object
strength_comfort_religion           2068 non-null object
petition_prayer_health              2068 non-null object
intercessory_prayers_health         2068 non-null object
died_2_year                         2068 non-null object
survival_time                       2068 non-null int64
depression                          2025 non-null object
anxiety      

### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,15)
df.info()

number of bad rows: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2068 entries, 0 to 2067
Data columns (total 60 columns):
site                                2068 non-null object
gender                              2053 non-null object
age                                 2068 non-null int64
race_ethnicity                      2059 non-null object
maritalstatus                       2068 non-null object
education                           2067 non-null object
lowhealthliteracy                   2068 non-null object
employment status                   2068 non-null object
healthinsurance                     2068 non-null object
strength_comfort_religion           2068 non-null object
petition_prayer_health              2068 non-null object
intercessory_prayers_health         2068 non-null object
died_2_year                         2068 non-null object
survival_time                       2068 non-null int64
depression                          2025 non-null object
anxiety           

In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 0


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[   2    2   65    3    2    3    2    2    2    3    2    2  726    5
    4    2    2    2   16 2065 2065   17    3    3   26  255  116  106
  155  308   42  246  186 1303    2    2    2    2    2    2    2    2
    2    3    3    2    2    2    2    2    2    2    2    2    2    2
    2    2    2]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 6: # < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [0, 1, 4, 6, 7, 8, 10, 11, 15, 16, 17, 34, 35, 36, 37, 38, 39, 40, 41, 42, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58]
i_category: [3, 5, 9, 13, 14, 22, 23, 43, 44]
variable type: [1. 1. 3. 2. 1. 2. 1. 1. 1. 2. 1. 1. 3. 2. 2. 1. 1. 1. 3. 3. 3. 3. 2. 2.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
#df = dfx
#df2 = df.copy()
#for i,col in enumerate(df.columns):
#    print(i,variable_type[i],col)
#    if variable_type[i] < 3: # binary or caterogy
#        df2[col] = df[col].fillna(df[col].mode().iloc[0])
#    else: # continuous
#        df2[col] = df[col].fillna(df[col].median())

In [21]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068 entries, 0 to 2067
Data columns (total 59 columns):
site                                2068 non-null object
gender                              2068 non-null object
age                                 2068 non-null int64
race_ethnicity                      2068 non-null object
maritalstatus                       2068 non-null object
education                           2068 non-null object
lowhealthliteracy                   2068 non-null object
employment status                   2068 non-null object
healthinsurance                     2068 non-null object
strength_comfort_religion           2068 non-null object
petition_prayer_health              2068 non-null object
intercessory_prayers_health         2068 non-null object
survival_time                       2068 non-null int64
depression                          2068 non-null object
anxiety                             2068 non-null object
stress                              2068

## Data Processing

### Attributes

In [22]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [23]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(2068, 80)
[[ 1.  1. 60. ... -1. -1.  1.]
 [ 1.  1. 61. ... -1. -1.  1.]
 [ 1.  1. 56. ... -1. -1. -1.]
 ...
 [ 1.  1. 73. ... -1. -1.  1.]
 [ 1. -1. 58. ... -1. -1.  1.]
 [ 1.  1. 72. ... -1. -1.  1.]]


### Target

In [24]:
y = np.array(dfy)
print(np.unique(y,return_counts=True))

# convert target to 0 and 1
y_new = np.ones(y.shape[0])
y_new[y == 'No'] = 0

# if target is already 0 and 1
#y_new = y #and delete lines below convert  
print(np.unique(y_new,return_counts=True))

(array(['No', 'Yes'], dtype=object), array([1945,  123]))
(array([0., 1.]), array([1945,  123]))


In [25]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')